In [41]:
import requests
import json
import rasterio as rio
from rasterio.io import MemoryFile
from rasterio.merge import merge
from rasterio.crs import CRS
from rasterio.warp import calculate_default_transform, reproject, Resampling
from PIL import Image
import io
import os

In [27]:
#example: https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x08/y26/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x08y26_2022--P1Y_final_canopy_cover_class.tif
#example: https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x24/y40/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x24y40_2022--P1Y_final_canopy_cover_class.tif
tile_url_pattern = "https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x{x}/y{y}/{year}--P1Y/ga_ls_mangrove_cover_cyear_3_x{x}y{y}_{year}--P1Y_final_canopy_cover_class.tif"


x_coordinates = [str(x).zfill(2) for x in range(8, 50)]
y_coordinates = [str(y).zfill(2) for y in range(10, 50)]

out_path = '../../../../../data/data/National_Layers/Australia_map_raster/'
os.makedirs(out_path, exist_ok=True)


In [28]:
#x_coordinates = ['08']
#y_coordinates = ['26', '27','30']

tiles = []
for x in x_coordinates:
        for y in y_coordinates:
            tile_url = tile_url_pattern.format(x=x, y=y, year=2022)
            response = requests.get(tile_url)
            if response.status_code == 200:
                tiles.append(tile_url)
                #print(f'Downloading tile: {x} / {y}')
                #tile_image = Image.open(io.BytesIO(response.content))
                #tile_image.save(out_path + f'AUS_mangrove_cover_2022_x{x}y{y}.tif')
tiles
                

['https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x08/y26/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x08y26_2022--P1Y_final_canopy_cover_class.tif',
 'https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x08/y27/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x08y27_2022--P1Y_final_canopy_cover_class.tif',
 'https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x08/y31/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x08y31_2022--P1Y_final_canopy_cover_class.tif',
 'https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x09/y26/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x09y26_2022--P1Y_final_canopy_cover_class.tif',
 'https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x09/y27/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x09y27_2022--P1Y_final_canopy_cover_class.tif',
 'https://data.dea.ga.gov.au/derivative/ga_ls_mangrove_cover_cyear_3/3-0-0/x09/y30/2022--P1Y/ga_ls_mangrove_cover_cyear_3_x09y30_2022--P1Y_final_canop

In [29]:
files_handler = [rio.open(i) for i in tiles]

In [30]:
memfile = MemoryFile()

merge(datasets=files_handler, # list of dataset objects opened in 'r' mode
    #bounds=tuple(gdf.set_crs("EPSG:4326").to_crs(files_handler[0].crs).total_bounds), # tuple
    nodata=None, # float
    dtype='uint16', # dtype
    method='first', # strategy to combine overlapping rasters
    dst_path=out_path + f'AUS_mangrove_cover_test.tif', # str or PathLike to save raster
    dst_kwds={'blockysize':512, 'blockxsize':512} # Dictionary
  )

In [32]:
combined_raster = rio.open(out_path + f'AUS_mangrove_cover_test.tif')

In [33]:
array = combined_raster.read()
array.shape

(1, 108800, 134400)

In [34]:
combined_raster.crs

CRS.from_epsg(3577)